### Import Libraries

In [ ]:
#rom utils.scanData import scan_audiofile
from utils.falseNegativeCount import countFN
#import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import utils.spect as spect
import pickle
import numpy as np
import pandas as pd
import os

### Define Helpful Functions

Here we define functions which will come in handy later.

In [ ]:
def plot_grid(X):
    
    """
    Plot an array of images in a grid.
    
    :param X: Array of images
    """
    
    num_per_row = 5
    rows = np.ceil(len(X) / num_per_row)

    plt.ion()
    fig = plt.figure(figsize=(5, 5))
    for i, image in enumerate(X):
        plt.subplot(rows, num_per_row, i + 1)
        plt.axis("off")
        plt.imshow(image)

    plt.subplots_adjust(hspace=0.1, wspace=0.1)
    plt.show()

### Set up File Paths

In [ ]:
# Store path to files
DATA_PATH = "E:/2018_sambay/deployment1/1073774637"

# Get list of audio file names
audio_files = os.listdir(DATA_PATH)

# Directory where results should be stored
WRITE_PATH = "C:/Users/mgoldwater/Desktop/JASASAMBAY"

csv_name = "results"

### Process the Data using the Model

In [ ]:
for file in audio_files:
    if file.split('.')[-1] == "wav":
        FILE_PATH = os.path.join(DATA_PATH, file)
        scan_audiofile(FILE_PATH, WRITE_PATH, 1, csv_name, batch_size=500)

### Load the Identified Multimodal Calls

In [ ]:
calls_df = pd.read_csv(os.path.join(WRITE_PATH, csv_name + ".csv"))

X = []
prev_filepath = None

for row in range(len(calls_df)):

    FILE_PATH = os.path.join(DATA_PATH, calls_df.loc[row].File.split("\\")[-1])

    # Don't reload the sound file if not necessary
    if FILE_PATH != prev_filepath:
        prev_filepath = FILE_PATH
        Fs_original, samples_norm = spect.get_and_normalize_sound(FILE_PATH)

    # Append channel dimension if none already
    if len(samples_norm.shape) == 1:
        samples_norm = samples_norm[:, np.newaxis]

    samples = samples_norm[calls_df.loc[row].StartSample:calls_df.loc[row].EndSample, calls_df.loc[row].Channel - 1]
    _, _, Zxx, _ = spect.my_stft(samples, Fs_original, window_N=31, window_overlap=5, NFFT=2 ** 8, DECIMATE_FACTOR=calls_df.loc[row].DecFactor)
    Zxx = Zxx[round(Zxx.shape[0] / 2):, :]
    freq = freq[:round(len(freq)/2)]
    spectro = 10 * np.log10(np.abs(Zxx) ** 2)

    # Cut to be 128X128
    length = spectro.shape[1]
    if length != 128:
        spectro = spectro[:, :length - (length - 128)]

    X.append(spectro)

X = np.asarray(X)

Plot some of the results.

In [ ]:
plot_grid(X[:60])